# 06: Build Ground Truth Validation Dataset

## Objective
Build a validation dataset to evaluate whether an LLM can correctly determine if a paper should be **included** or **excluded** from a Cochrane systematic review.

## Revised Label Logic
- **Included (label=1)**: ALL papers referenced in any Cochrane review, regardless of their
  internal categorization (included, excluded, awaiting, ongoing). If human reviewers
  mentioned it, it was relevant enough to consider.
- **Excluded (label=0)**: Papers retrieved by the review's PubMed search strategy but
  NOT referenced in any Cochrane review. These are true negatives — the search found them
  but reviewers did not include them.

## What the LLM needs to decide
Given:
- **Review context** (title and abstract of the Cochrane systematic review)
- **Paper abstract** (a candidate paper being screened)

Predict:
- **Label = 1** → Paper should be INCLUDED in the review
- **Label = 0** → Paper should be EXCLUDED from the review

## Data structure
Each row in the validation set contains:
1. `review_doi` - DOI of the Cochrane systematic review
2. `review_title` - Title of the Cochrane systematic review
3. `review_abstract` - Abstract/objective of the review (screening criteria)
4. `cochrane_group` - Cochrane Review Group (for filtering)
5. `paper_title` - Title of the candidate paper
6. `paper_abstract` - Abstract of the candidate paper
7. `label` - Ground truth (1=included, 0=excluded)
8. `source` - Origin of the paper ("cochrane_ref" or "pubmed_search")

## Input Files
- `Data/referenced_paper_abstracts.csv` — Papers referenced in Cochrane reviews (notebook 04) → **all label=1**
- `Data/pubmed_excluded_abstracts.csv` — PubMed search-only papers (notebook 05) → **all label=0**
- `Data/cochrane_pubmed_abstracts.csv` — Cochrane review abstracts (notebook 00)
- `Data/review_metadata.csv` — Review metadata including Cochrane group

## Output
- `Data/ground_truth_validation_dataset.csv` - Full validation dataset with `cochrane_group` column for filtering

In [1]:
# Install required packages
%pip install -q pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
# =============================================================================
# Load All Data Sources
# =============================================================================
from pathlib import Path
import pandas as pd

notebook_dir = Path.cwd()
project_root = notebook_dir if (notebook_dir / "Data").exists() else notebook_dir.parent
DATA_DIR = project_root / "Data"

# Input files
INCLUDED_CSV  = DATA_DIR / "referenced_paper_abstracts.csv"   # from notebook 04
EXCLUDED_CSV  = DATA_DIR / "pubmed_excluded_abstracts.csv"     # from notebook 05
REVIEWS_CSV   = DATA_DIR / "cochrane_pubmed_abstracts.csv"     # from notebook 00
META_CSV      = DATA_DIR / "review_metadata.csv"               # from notebook 03
OUTPUT_CSV    = DATA_DIR / "ground_truth_validation_dataset.csv"

# --- Included papers (ALL Cochrane references) ---
print("Loading included papers (Cochrane references)...")
included_raw = pd.read_csv(INCLUDED_CSV)
print(f"  Total rows: {len(included_raw):,}")
print(f"  Columns: {included_raw.columns.tolist()}")
assert 'review_doi' in included_raw.columns, "ERROR: review_doi missing! Re-run notebook 04."

# --- Excluded papers (PubMed search-only) ---
print("\nLoading excluded papers (PubMed search results)...")
excluded_raw = pd.read_csv(EXCLUDED_CSV)
print(f"  Total rows: {len(excluded_raw):,}")
print(f"  Columns: {excluded_raw.columns.tolist()}")

# --- Review abstracts ---
print("\nLoading Cochrane review abstracts...")
reviews = pd.read_csv(REVIEWS_CSV)
print(f"  Total review abstracts: {len(reviews):,}")

# --- Review metadata ---
meta = pd.read_csv(META_CSV)
print(f"  Review metadata: {len(meta):,} reviews")

Loading included papers (Cochrane references)...
  Total rows: 47,518
  Columns: ['study_id', 'review_doi', 'category', 'original_title', 'original_authors', 'original_year', 'pmid', 'doi', 'matched_title', 'matched_authors', 'matched_year', 'abstract', 'match_method']

Loading excluded papers (PubMed search results)...
  Total rows: 279,373
  Columns: ['review_doi', 'pmid', 'title', 'abstract', 'authors', 'year', 'doi']

Loading Cochrane review abstracts...
  Total review abstracts: 17,298
  Review metadata: 16,588 reviews


In [3]:
# =============================================================================
# Prepare Included Papers (label=1)
# =============================================================================
# ALL papers referenced in Cochrane reviews — regardless of internal category
# (included, excluded, awaiting, ongoing) — are label=1.

print("INCLUDED PAPERS (label=1)")
print("=" * 60)

# Filter to rows with usable abstracts
included = included_raw[
    included_raw['abstract'].notna() &
    (included_raw['abstract'].str.len() > 50)
].copy()
print(f"With abstracts (>50 chars): {len(included):,}")

# Deduplicate (study_id, review_doi)
included = included.drop_duplicates(subset=['study_id', 'review_doi'], keep='first')
print(f"After dedup: {len(included):,}")

# Standardize columns for merge
included_clean = included[['review_doi', 'original_title', 'abstract']].rename(columns={
    'original_title': 'paper_title',
    'abstract': 'paper_abstract',
})
included_clean['label'] = 1
included_clean['source'] = 'cochrane_ref'

# Also keep PMID as a unique paper ID where available
if 'pmid' in included.columns:
    included_clean['pmid'] = included['pmid'].values

print(f"\nIncluded papers prepared: {len(included_clean):,}")
print(f"  Unique reviews: {included_clean['review_doi'].nunique():,}")
print(f"  Original categories: {dict(included['category'].value_counts())}")

INCLUDED PAPERS (label=1)
With abstracts (>50 chars): 42,869
After dedup: 42,840

Included papers prepared: 42,840
  Unique reviews: 1,231
  Original categories: {'excluded': np.int64(26954), 'included': np.int64(14738), 'awaiting': np.int64(678), 'ongoing': np.int64(470)}


In [4]:
# =============================================================================
# Prepare Excluded Papers (label=0)
# =============================================================================
# Papers found by PubMed search strategies but NOT in any Cochrane review.

print("EXCLUDED PAPERS (label=0)")
print("=" * 60)

# Excluded already filtered to abstract > 50 chars in notebook 05
excluded = excluded_raw.copy()
print(f"Excluded papers loaded: {len(excluded):,}")

# Standardize columns
excluded_clean = excluded[['review_doi', 'title', 'abstract']].rename(columns={
    'title': 'paper_title',
    'abstract': 'paper_abstract',
})
excluded_clean['label'] = 0
excluded_clean['source'] = 'pubmed_search'

if 'pmid' in excluded.columns:
    excluded_clean['pmid'] = excluded['pmid'].values

print(f"Excluded papers prepared: {len(excluded_clean):,}")
print(f"  Unique reviews: {excluded_clean['review_doi'].nunique():,}")

# =============================================================================
# Combine Included + Excluded
# =============================================================================

print("\n" + "=" * 60)
print("COMBINING DATASETS")
print("=" * 60)

combined = pd.concat([included_clean, excluded_clean], ignore_index=True)
print(f"Combined total: {len(combined):,}")
print(f"  Included (label=1): {(combined['label'] == 1).sum():,}")
print(f"  Excluded (label=0): {(combined['label'] == 0).sum():,}")

EXCLUDED PAPERS (label=0)
Excluded papers loaded: 279,373
Excluded papers prepared: 279,373
  Unique reviews: 2,147

COMBINING DATASETS
Combined total: 322,213
  Included (label=1): 42,840
  Excluded (label=0): 279,373


In [5]:
# =============================================================================
# Join with Review Context (title, abstract, cochrane_group)
# =============================================================================

print("JOINING WITH REVIEW CONTEXT")
print("=" * 60)

# Prepare review context (deduplicated)
review_groups = meta[['doi', 'cochrane_group']].rename(columns={'doi': 'review_doi'})

reviews_clean = reviews[['doi', 'title', 'abstract']].rename(columns={
    'doi': 'review_doi',
    'title': 'review_title',
    'abstract': 'review_abstract',
}).drop_duplicates(subset='review_doi', keep='first')

reviews_clean = reviews_clean.merge(review_groups, on='review_doi', how='left')
print(f"Unique reviews with context: {len(reviews_clean):,}")

# Join combined papers with review context
ground_truth = combined.merge(reviews_clean, on='review_doi', how='inner')
print(f"Papers matched to reviews: {len(ground_truth):,}")

# Filter to reviews with meaningful abstracts
ground_truth = ground_truth[
    ground_truth['review_abstract'].notna() &
    (ground_truth['review_abstract'].str.len() > 100)
]
print(f"With review abstract > 100 chars: {len(ground_truth):,}")

# Final column selection
ground_truth = ground_truth[[
    'review_doi',
    'review_title',
    'review_abstract',
    'cochrane_group',
    'paper_title',
    'paper_abstract',
    'label',
    'source',
]].copy()

print(f"\nVALIDATION DATASET STRUCTURE")
print("=" * 60)
print(f"Total examples: {len(ground_truth):,}")
print(f"\nLabel distribution:")
print(f"  Included (label=1): {(ground_truth['label'] == 1).sum():,}")
print(f"  Excluded (label=0): {(ground_truth['label'] == 0).sum():,}")
print(f"  Ratio (excl/incl):  {(ground_truth['label'] == 0).sum() / max((ground_truth['label'] == 1).sum(), 1):.1f}x")
print(f"\nUnique reviews: {ground_truth['review_doi'].nunique():,}")
print(f"Cochrane groups: {ground_truth['cochrane_group'].nunique():,}")

print("\n" + "=" * 60)
print("SAMPLE ROW (what the LLM will see):")
print("=" * 60)
sample = ground_truth.iloc[0]
print(f"\nREVIEW TITLE: {sample['review_title'][:100]}...")
print(f"\nREVIEW ABSTRACT: {sample['review_abstract'][:300]}...")
print(f"\nPAPER TITLE: {sample['paper_title']}")
print(f"\nPAPER ABSTRACT: {sample['paper_abstract'][:300]}...")
print(f"\nLABEL: {sample['label']} ({'INCLUDED' if sample['label']==1 else 'EXCLUDED'})")
print(f"SOURCE: {sample['source']}")

JOINING WITH REVIEW CONTEXT
Unique reviews with context: 16,617
Papers matched to reviews: 322,213
With review abstract > 100 chars: 322,213

VALIDATION DATASET STRUCTURE
Total examples: 322,213

Label distribution:
  Included (label=1): 42,840
  Excluded (label=0): 279,373
  Ratio (excl/incl):  6.5x

Unique reviews: 3,254
Cochrane groups: 74

SAMPLE ROW (what the LLM will see):

REVIEW TITLE: Acupuncture for smoking cessation....

REVIEW ABSTRACT: Acupuncture is promoted as a treatment for smoking cessation, and is believed to reduce withdrawal symptoms. The objective of this review is to determine the effectiveness of acupuncture in smoking cessation in comparison with: a) sham acupuncture b) other interventions c) no intervention. We search...

PAPER TITLE: Laser acupuncture for adolescent smokers - a randomized double-blind controlled trial

PAPER ABSTRACT: A double blind, randomized, placebo-controlled clinical study was conducted to evaluate the efficacy of laser acupuncture trea

In [6]:
# =============================================================================
# Save Validation Dataset
# =============================================================================

ground_truth.to_csv(OUTPUT_CSV, index=False)

print("GROUND TRUTH VALIDATION DATASET COMPLETE")
print("=" * 60)
print(f"\n✓ Saved to {OUTPUT_CSV.name}")
print(f"  Total examples:    {len(ground_truth):,}")
print(f"  Included (label=1): {(ground_truth['label'] == 1).sum():,}")
print(f"  Excluded (label=0): {(ground_truth['label'] == 0).sum():,}")
print(f"  Unique reviews:    {ground_truth['review_doi'].nunique():,}")

print(f"\nCochrane groups:")
print(ground_truth['cochrane_group'].value_counts().to_string())

print(f"\nSource breakdown:")
print(ground_truth['source'].value_counts().to_string())

print(f"\n✓ Ready for notebook 07 (LLM evaluation)!")

GROUND TRUTH VALIDATION DATASET COMPLETE

✓ Saved to ground_truth_validation_dataset.csv
  Total examples:    322,213
  Included (label=1): 42,840
  Excluded (label=0): 279,373
  Unique reviews:    3,254

Cochrane groups:
cochrane_group
Eyes and Vision                                    29036
Acute Respiratory Infections                       24799
Oral Health                                        20826
Kidney and Transplant                              17259
Drugs and Alcohol                                  12260
Gynaecology and Fertility                          11228
Tobacco Addiction                                  11012
Infectious Diseases                                 9037
Gynaecological Cancer                               8043
Musculoskeletal                                     7796
Airways                                             7130
Heart                                               6922
Stroke                                              6882
Injuries              